# Panel data regression model
For the empirical part of my thesis project for master of Business Economics, I have used Panel data regression model with rnadom effects. I used linearmodels library. First, the data is normalized to be prepared for use in regresion model. Then the model is fitted.

In [1]:
# Import Libraries
import numpy as np
import pandas as pd
import statsmodels as sm
from linearmodels import PanelOLS
from linearmodels import RandomEffects

In [2]:
# Import data from Excel file
Data = pd.read_excel('Content analysis.xlsx',sheet_name='Final', na_values=np.nan, header=1, index_col='Bank')
Data.drop(columns='BankID', inplace=True)
Data.columns

Index(['Country', 'Year', 'Innovation-BoD', 'Innovation-CEO', 'Innovation',
       'Risk-BoD', 'Risk-CEO', 'Risk_taking', 'Islamicity-BoD',
       'Islamicity-CEO', 'Islamicity', 'Orientation-BoD', 'Orientation_CEO',
       'Orientation', 'ROA', 'ROE', 'firm_age', 'AssetSize_dollar',
       'NPL_ratio', 'Window', 'FullyFledged', 'Total_IB', 'GDP_growth (%)',
       'Inflation', 'Real_interest_rate (%)', 'FDI_inflow (USD)'],
      dtype='object')

In [4]:
# Normalization
Bank_mean = Data.groupby('Bank').mean().drop(columns='Year')
Bank_std = Data.groupby('Bank').std().drop(columns='Year')
Data_merged = pd.merge(Data, Bank_mean, on='Bank', suffixes=('','_mb'))
Data_merged = pd.merge(Data_merged, Bank_std, on='Bank', suffixes=('','_sb'))

In [5]:
Norm_Cols = ['Innovation', 'Risk_taking', 'Islamicity', 'Orientation', 'ROA', 'ROE', 'firm_age', 'AssetSize_dollar', 
             'NPL_ratio', 'Window', 'FullyFledged', 'Total_IB', 'GDP_growth (%)', 'Inflation', 'Real_interest_rate (%)', 
             'FDI_inflow (USD)'
            ]

def normalize(col, mean, stdev): # function to colculate the normalized value for each column
    norm = (col - mean)/stdev
    return norm

In [6]:
Data_normal = Data.copy()
for col in Norm_Cols:
    Data_normal[col] = normalize(Data_merged[col], Data_merged[col+'_mb'], Data_merged[col+'_sb'])

In [7]:
Data_normal

,Country,Year,Innovation-BoD,Innovation-CEO,Innovation,Risk-BoD,Risk-CEO,Risk_taking,Islamicity-BoD,Islamicity-CEO,...,firm_age,AssetSize_dollar,NPL_ratio,Window,FullyFledged,Total_IB,GDP_growth (%),Inflation,Real_interest_rate (%),FDI_inflow (USD)
Bank,,,,,,,,,,,,,,,,,,,,,
Abu Dhabi Islamic Bank,UAE,2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.460593,-1.416479,0.846314,-1.178511,1.333333,-1.000000,-0.984683,1.157489,NaN,-0.576555
Abu Dhabi Islamic Bank,UAE,2011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.095445,-1.416479,1.372136,-1.178511,1.333333,-1.000000,1.593251,1.233951,NaN,-1.981078
Abu Dhabi Islamic Bank,UAE,2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.730297,-0.899655,1.260405,-1.178511,1.333333,-1.000000,0.409815,-0.003545,NaN,0.080911
Abu Dhabi Islamic Bank,UAE,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.365148,-0.038283,0.495014,-0.471405,-0.666667,-1.000000,0.685015,-0.355140,NaN,0.250224
Abu Dhabi Islamic Bank,UAE,2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.306266,-0.715858,0.235702,-0.666667,0.000000,0.312879,-0.371782,NaN,1.366057
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Warba Bank,Kuwait,2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.204124,-0.375727,-0.452051,NaN,0.661438,0.661438,-0.403736,-0.585249,0.410024,-0.464498
Warba Bank,Kuwait,2015,0.004702,NaN,-0.864529,0.001567,NaN,-1.352961,0.014107,NaN,...,0.204124,-0.170971,0.292423,NaN,0.661438,0.661438,-0.382518,-1.875912,2.096010,-0.619166
Warba Bank,Kuwait,2016,0.005929,NaN,-0.326771,0.001976,NaN,-0.734215,0.010870,NaN,...,0.612372,0.279492,0.565962,NaN,0.661438,0.661438,0.157790,-0.590626,0.428558,-0.613493


In [9]:
Data_prep = Data_normal.reset_index().set_index(['Country', 'Bank'])
Data_prep.loc[:, Norm_Cols].groupby('Bank').describe().transpose()

Bank                    Abu Dhabi Islamic Bank  \
Innovation       count            4.000000e+00   
                 mean             1.153591e-16   
                 std              1.000000e+00   
                 min             -1.010380e+00   
                 25%             -5.244890e-01   
...                                        ...   
FDI_inflow (USD) min             -1.981078e+00   
                 25%             -5.765546e-01   
                 50%              1.134663e-01   
                 75%              7.534836e-01   
                 max              1.366057e+00   

Bank                    Abu Dhabi Islamic Bank (National Bank for Development)-Egypt  \
Innovation       count                                           2.000000              
                 mean                                            0.000000              
                 std                                             1.000000              
                 min                                            -0.707107              
                 25%                                            -0.353553              
...                                                                   ...              
FDI_inflow (USD) min                                            -2.224841              
                 25%                                            -0.400172              
                 50%                                             0.459273              
                 75%                                             0.590858              
                 max                                             1.054096              

Bank                        Agrobank  Ahli United Bank  Ajman Islamic Bank  \
Innovation       count  4.000000e+00      9.000000e+00            0.000000   
                 mean  -1.110223e-16      2.097088e-16                 NaN   
                 std    1.000000e+00      1.000000e+00                 NaN   
                 min   -1.120234e+00     -1.406588e+00                 NaN   
                 25%   -6.796121e-01     -6.283536e-01                 NaN   
...                              ...               ...                 ...   
FDI_inflow (USD) min   -8.053982e-01     -9.314596e-01           -1.981078   
                 25%   -5.292479e-01     -6.803492e-01           -0.576555   
                 50%   -3.245086e-01     -5.152873e-01            0.113466   
                 75%    2.047393e-01      2.624539e-01            0.753484   
                 max    1.454415e+00      1.760325e+00            1.366057   

Bank                    Al Baraka Bank  Al Baraka Bank Egypt SAE-Egypt  \
Innovation       count    7.000000e+00                    6.000000e+00   
                 mean     1.744636e-16                    5.551115e-17   
                 std      1.000000e+00                    1.000000e+00   
                 min     -1.262000e+00                   -1.636951e+00   
                 25%     -8.747022e-01                   -4.376731e-01   
...                                ...                             ...   
FDI_inflow (USD) min     -7.581653e-01                   -2.224841e+00   
                 25%     -6.814381e-01                   -4.001720e-01   
                 50%     -3.748649e-01                    4.592726e-01   
                 75%      4.692043e-01                    5.908576e-01   
                 max      2.334330e+00                    1.054096e+00   

Bank                    Al Baraka Bank Sudan  \
Innovation       count              0.000000   
                 mean                    NaN   
                 std                     NaN   
                 min                     NaN   
                 25%                     NaN   
...                                      ...   
FDI_inflow (USD) min               -1.367995   
                 25%               -0.890834   
                 50%                0.212111   
                 75%                

In [11]:
year = Data_prep['Year']
Data_set = Data_prep.reset_index().set_index(['Bank', 'Year'])
exog_vars = ['Innovation', 'Risk_taking', 'Islamicity', 'Orientation', 'firm_age', 'AssetSize_dollar', 
             'NPL_ratio', 'Window', 'FullyFledged', 'Total_IB', 'GDP_growth (%)', 'Inflation', 'Real_interest_rate (%)', 
             'FDI_inflow (USD)']
exog = Data_set[exog_vars]
mod = RandomEffects(Data_set['ROA'], exog)
re_res = mod.fit()

In [12]:
print(re_res)

                        RandomEffects Estimation Summary                        
Dep. Variable:                    ROA   R-squared:                        0.2568
Estimator:              RandomEffects   R-squared (Between):              0.3282
No. Observations:                  86   R-squared (Within):               0.2453
Date:                Mon, Jul 06 2020   R-squared (Overall):              0.2568
Time:                        18:14:55   Log-likelihood                   -95.390
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      1.7766
Entities:                          10   P-value                           0.0591
Avg Obs:                       8.6000   Distribution:                   F(14,72)
Min Obs:                       4.0000                                           
Max Obs:                       12.000   F-statistic (robust):             1.7766
                            